In [24]:
import os
import re
import numpy as np
from selenium import webdriver
import pandas as pd
import numpy as np
from time import sleep
from selenium.common.exceptions import TimeoutException

In [25]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless=new")

In [26]:
os.environ['PATH'] += r"/usr/local/bin/"
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(5) 
# driver.maximize_window()
sleep(5)

In [27]:
def get_data(text):
    reg_template = {"Area":'Diện tích:\s*(\d+)',
                "Room":'Phòng ngủ:\s*(\d+)',
                "WC":'Phòng WC:\s*(\d+)',
                "Location":'Địa chỉ:\s*(.+)'}
    list_params = []
    for i,name in enumerate(reg_template):
        x = re.search(r'{}'.format(reg_template[name]),text)
        if x:
            list_params.append(x.group(1))
        else:
            list_params.append(np.nan)
    return list_params



In [28]:
base_url = "https://batdongsan.vn/ban-nha/"
report =    pd.DataFrame(columns=['Title', 'Price', 'Links',\
            'Diện tích (m2)', 'Số phòng ngủ','Số phòng WC', 'Địa chỉ'\
            ,'Description'])

n_iter = 1
for i in range(n_iter):
    url = base_url + "p{}".format(i+1)
    print(f"------Start crawl page {i+1}------")
    # Duyệt page thứ i
    try :
        print("\t------Loading page------")
        driver.get(url)
    except TimeoutException:
        pass
    # Lấy element là các item trong một page (20 items)
    try:
        print("\t------Getting links to 20 items------")
        elements = driver.find_elements('css selector','.uk-width-medium-7-10 .name [href]')
    except:
        pass
    
    # Trích xuất dẫn đến mô tảchi tiết của từng item
    links = [element.get_attribute('href') for element in elements]
    # Duyệt qua từng item
    n = len(links)
    data = []
    print("\t------Start extracting information------")
    for j in range(n):
        # Mở link dẫn đến item
        try:
            driver.get(links[j])
        except Exception as e:
            pass
        # Lấy ra những thông tin cần thiết (Title, Price, Số Phòng ngủ, Số WC, Diện tích, Description...etc)
        try:
            # Cho page 1s để load những element cần thiết - tránh trường hợp tìm không ra
            sleep(1)
            # Tìm các element chứa những thông tin cần thiết
            title_ele = driver.find_element('css selector','.uk-panel-title')
            price_ele = driver.find_element('css selector','.meta .price')
            params_ele = driver.find_element('css selector',".param")
            content_ele = driver.find_element('css selector','.item .body .content')
            # Trích xuất văn bản từ web element
            title = title_ele.text
            price = price_ele.text
            params_ = get_data(params_ele.text)
            params_.append(content_ele.text)
            # Gộp các thông tin thành một list tmp_list theo form la các feature trong data frame report đã khai báo
            tmp_list = [title,price,links[j]]
            tmp_list = tmp_list + params_
            data.append(tmp_list)
        except:
            pass
    # Xếp chồng các phần tử trong list data trước đó để tạo thành ma trận với mỗi hàng là một mẫu trong bộ dữ liệu
    matrix_data = np.vstack([item for item in data]) 
    # Tạo ra data_page là dataframe chứa dữ liệu của một trang và concat vào dataframe report
    data_page = pd.DataFrame(matrix_data,columns=report.columns)
    
    report = pd.concat([report,data_page],axis=0)
    report.set_index(np.arange(len(report)),inplace=True)
    # Sao lưu lại data vào file output.csv tránh trường hợp bị mất dữ liệu trong quá trình cào
    report.to_csv('output.csv',sep="\t",index=False)
    print(f"------Page{i} - Done!------\n\n")

report

------Start crawl page 1------
	------Loading page------
	------Getting links to 20 items------
	------Start extracting information------
------Page0 - Done!------




,Title,Price,Links,Diện tích (m2),Số phòng ngủ,Số phòng WC,Địa chỉ,Description
0,,3899000000 tỷ,https://batdongsan.vn/chua-toi-30trm2-hang-ngo...,150,2,1,Thành phố Hồ Chí Minh,Mô tả:\n+ Thông số cực đẹp 5 x 30m (150m2)\...
1,,5.5 tỷ,https://batdongsan.vn/ban-nha-hxh-au-co-phuong...,51,nan,nan,ÂU CƠ PHƯỜNG 9 TÂN BÌNH,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ..."
2,,4.6 tỷ,https://batdongsan.vn/sat-mat-tien-phan-dang-l...,45,2,2,nan,"- Kết cấu: 1 trệt 1 lầu BTCT; 2 PN, 2 WC. Phòn..."
3,,7.35 tỷ,https://batdongsan.vn/chu-gap-ban-truoc-tet-le...,41,nan,nan,LÊ HỒNG PHONG PHƯỜNG 2 QUẬN 5,"Mô tả:\n- Diện tích công nhận 41m, thực tế 66m..."
4,,Thỏa thuận,https://batdongsan.vn/luy-ban-bichtan-phu-dien...,96,2,1,nan,- Diện tích 96m2 ngang 4.3m dài 22.5m .\n- sổ ...
5,,3.95 tỷ,https://batdongsan.vn/ban-nha-trong-ngo-le-tro...,40,4,3,"La Khê, Hà Đông",- Vị trí khu vực trung tâm phường La Khê- Hà Đ...
6,,3.95 tỷ,https://batdongsan.vn/ban-nha-trong-ngo-le-tro...,40,4,3,"La Khê, Hà Đông",- Vị trí khu vực trung tâm phường La Khê- Hà Đ...
7,,7.95 tỷ,https://batdongsan.vn/ban-nha-pho-hoang-van-th...,59,nan,nan,nan,+ Chủ nhà cần bán gấp nhà phố Hoàng Văn Thái -...
8,,Thỏa thuận,https://batdongsan.vn/ban-nha-moi-dep-7-x-13-c...,nan,3,3,nan,"✅ BÁN NHÀ MỚI ĐẸP 7 X 13, CHỈ 3.5 TỶ, HẺM XE H..."
9,,17 tỷ,https://batdongsan.vn/sieu-pham-chao-ban-can-b...,425,6,6,"Lê Hồng Phong, Ngô Quyền, Hải Phòng",SIÊU PHẨM Chào bán căn Biệt Thự mặt đường Lê H...
